## DEAD

Fitting a model (or two) based on our proposal.

- **Goal:** Build a model to predict sales in a month for any given store.
- **Response Variable:** Monthly Sales
- **Possible Features:** store, month, county, population stuff, proximity stuff, alcohol categories

In [9]:
import duckdb as db 
con = db.connect()
import pandas as pd 
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# MAIN TABLE
con.execute("""
        DROP TABLE IF EXISTS sales;
        CREATE TABLE sales AS 
        SELECT EXTRACT(MONTH FROM date) AS month, EXTRACT (YEAR FROM date) AS year,
            store, city, county, 
            category_name AS category, sale_bottles AS bottles, sale_dollars AS dollars
        FROM read_parquet('../data/iowa_liquor_2023_2025.parquet');
""")
sales = con.execute("SELECT * FROM sales").df()

# POPULATION
con.execute(
"""
        DROP TABLE IF EXISTS population;
        CREATE TABLE population AS
        SELECT name AS county, year_1 AS year, popestimate AS population, over21, propOver21, median_age_tot AS median_age, (POPEST_MALE / (POPEST_FEM + POPEST_MALE)) AS pct_male
        FROM read_csv_auto('../data/pop.csv');
"""
)
pop = con.execute("SELECT * FROM population").df()

# PROXIMITY
con.execute(
"""
        DROP TABLE IF EXISTS proximity;
        CREATE TABLE proximity AS
        SELECT county
        FROM read_csv_auto('../data/proximity.csv');
"""
)
prox = con.execute("SELECT * FROM proximity").df()




In [138]:
socioeconomic = pd.read_csv("../data/HDPulse_socioeconomic_data.csv")

socioeconomic["County"] = socioeconomic["County"].str.replace(r"\s+County$", "", regex=True)
socioeconomic.rename(columns={
    "County": "county",
    "FIPS": "fips",
    "% People (Education: Less Than 9th Grade)": "pct_less_than_9th_grade",
    "People (Education: Less Than 9th Grade)": "less_than_9th_grade",
    "% People (Unemployed)": "pct_unemployed",
    "People (Unemployed)": "unemployed",
    "% People (Below Poverty)": "pct_below_poverty",
    "People (Below Poverty)": "below_poverty",
    "% People (<150% Of Poverty)": "pct_below_150pct_poverty",
    "People (<150% Of Poverty)": "below_150pct_poverty",
    "% Families (Below Poverty)": "pct_families_below_poverty",
    "Families (Below Poverty)": "families_below_poverty",
    "% People (White)": "pct_white",
    "People (White)": "white",
    "% People (Hispanic)": "pct_hispanic",
    "People (Hispanic)": "hispanic",
    "% People (Foreign Born)": "pct_foreign_born",
    "People (Foreign Born)": "foreign_born",
    "% People (Black)": "pct_black",
    "People (Black)": "black",
    "% People (API)": "pct_api",
    "People (API)": "api",
    "% People (AI/AN)": "pct_aian",
    "People (AI/AN)": "aian",
    "% Households (language Isolation)": "pct_language_isolation",
    "Households (language Isolation)": "language_isolation",
    "% People(Education: Less Than High School)": "pct_less_than_high_school",
    "People(Education: Less Than High School)": "less_than_high_school",
    "% People (Education: At Least Bachelor's Degree)": "pct_at_bachelor_degree"
}, inplace=True)
socioeconomic.head()

,county,fips,pct_less_than_9th_grade,less_than_9th_grade,pct_unemployed,unemployed,pct_below_poverty,below_poverty,pct_below_150pct_poverty,below_150pct_poverty,pct_families_below_poverty,families_below_poverty,pct_white,white,pct_hispanic,hispanic,pct_foreign_born,foreign_born,pct_black,black,pct_api,api,pct_aian,aian,pct_language_isolation,language_isolation,pct_less_than_high_school,less_than_high_school,pct_at_bachelor_degree
0,Adair,19001.0,2.1,110.0,4.2,162.0,10.3,753.0,21.8,1591.0,5.0,92.0,94.9,7091.0,2.9,214.0,1.6,120.0,1.4,101.0,0.5,34.0,0.4,27.0,0.0,0.0,5.5,287.0,20.0
1,Adams,19003.0,2.6,68.0,2.9,53.0,9.5,339.0,17.7,629.0,3.6,35.0,95.4,3475.0,0.7,27.0,0.2,9.0,0.4,13.0,0.0,0.0,0.5,20.0,0.3,5.0,7.4,197.0,24.5
2,Allamakee,19005.0,3.0,293.0,3.6,252.0,12.1,1657.0,19.8,2716.0,9.4,350.0,90.4,12690.0,8.7,1225.0,6.7,937.0,1.8,246.0,0.5,76.0,0.8,115.0,4.4,265.0,7.5,721.0,19.9
3,Appanoose,19007.0,5.5,480.0,6.9,375.0,20.4,2468.0,31.9,3851.0,10.8,346.0,95.1,11641.0,1.7,212.0,1.2,150.0,1.0,121.0,0.4,53.0,0.1,7.0,1.0,51.0,10.3,907.0,19.3
4,Audubon,19009.0,2.7,108.0,1.9,52.0,11.8,649.0,20.9,1147.0,7.1,109.0,95.2,5351.0,2.0,115.0,0.9,48.0,0.3,15.0,0.1,7.0,0.2,12.0,0.9,23.0,8.0,322.0,20.6


In [159]:
socioeconomic["pct_high_school_lower"] = socioeconomic["pct_less_than_high_school"] + socioeconomic["pct_less_than_9th_grade"]

In [13]:
prox = pd.DataFrame(prox.value_counts())
prox.rename(columns={'count': 'store_count'}, inplace=True)
prox.reset_index(inplace=True)
prox.head()

,county,store_count
0,POLK,317
1,LINN,168
2,SCOTT,112
3,BLACK HAWK,108
4,JOHNSON,93


In [14]:
sales.columns

Index(['month', 'year', 'store', 'city', 'county', 'category', 'bottles',
       'dollars'],
      dtype='object')

## Creating the Dataset

First, I am going to engineer the category column a little bit to use as features. Knowing which alcohol sells the best is could be useful for telling Booze R Us what they should buy in order to increase profits.

In [15]:
con.execute("""
    CREATE OR REPLACE TABLE sales AS
    SELECT *,
        CASE
            WHEN category ILIKE '%VODKA%' THEN 'Vodka'
            WHEN category ILIKE '%WHISK%' THEN 'Whiskey'
            WHEN category ILIKE '%TEQUILA%' OR category ILIKE '%MEZCAL%' THEN 'Tequila'
            WHEN category ILIKE '%RUM%' THEN 'Rum'
            ELSE 'Other'
        END AS super_category
    FROM sales
""")
sales = con.execute("SELECT * FROM sales").df()

In [16]:
sales.head()

,month,year,store,city,county,category,bottles,dollars,super_category
0,1,2023,4829,DES MOINES,POLK,100% AGAVE TEQUILA,12,261.00,Tequila
1,1,2023,4829,DES MOINES,POLK,AMERICAN VODKAS,60,418.80,Vodka
2,1,2023,4829,DES MOINES,POLK,IMPORTED FLAVORED VODKA,24,358.56,Vodka
3,1,2023,4829,DES MOINES,POLK,CREAM LIQUEURS,12,306.00,Other
4,1,2023,4829,DES MOINES,POLK,SPICED RUM,60,1124.40,Rum


Now I need to agreggate to create our appropriate observational units: monthly sales per store.

- Dollars (our response variable) will be summed. 
- Category will be made into new columns representing the distribution of category sales
    - e.g. 70% tequila, 20% vodkas, etc.
    - we will not use total bottles because this would be almost perfectly collinear 
    - answer questions like: 'what liquor should we sell more/less of?'

In [17]:
monthly_sales = con.execute(
"""
    WITH month_totals AS (
        SELECT
            year,
            month,
            county,
            SUM(dollars) AS revenue
        FROM sales
        GROUP BY year, month, county
    ), category_totals AS (
        SELECT
            year,
            month,
            county,
            super_category,
            SUM(dollars) AS category_sales
        FROM sales
        GROUP BY year, month, county, super_category
    )
    SELECT
        mt.year,
        mt.month,
        mt.county,
        ROUND(SUM(CASE WHEN ct.super_category = 'Vodka' THEN ct.category_sales ELSE 0 END) / mt.revenue, 2) AS vodka_ptc,
        ROUND(SUM(CASE WHEN ct.super_category = 'Whiskey' THEN ct.category_sales ELSE 0 END) / mt.revenue, 2) AS whiskey_ptc,
        ROUND(SUM(CASE WHEN ct.super_category = 'Tequila' THEN ct.category_sales ELSE 0 END) / mt.revenue, 2) AS tequila_ptc,
        ROUND(SUM(CASE WHEN ct.super_category = 'Rum' THEN ct.category_sales ELSE 0 END) / mt.revenue, 2) AS rum_ptc,
        ROUND(SUM(CASE WHEN ct.super_category = 'Other' THEN ct.category_sales ELSE 0 END) / mt.revenue, 2) AS other_ptc,
        mt.revenue
    FROM month_totals mt
    LEFT JOIN category_totals ct
        ON mt.year = ct.year
        AND mt.month = ct.month
        AND mt.county = ct.county
    GROUP BY mt.year, mt.month, mt.county, mt.revenue
"""
).fetchdf()


In [18]:
monthly_sales.head(2)

,year,month,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue
0,2023,2,DES MOINES,0.24,0.38,0.07,0.09,0.21,351538.01
1,2023,2,FRANKLIN,0.16,0.52,0.11,0.08,0.13,42850.42


Now I will join with our other datasets, proximity and population. Using an inner join because it still leaves plenty of complete data for modelling. 

In [160]:
df = con.execute(
    """
        SELECT sales.*, 
            pop.population, pop.over21, pop.propOver21, pop.median_age, pop.pct_male, prox.store_count, socioeconomic.*
        FROM monthly_sales sales
        JOIN pop
            ON LOWER(sales.county) = LOWER(pop.county) AND sales.year = pop.year
        JOIN prox
            on LOWER(pop.county) = LOWER(prox.county)
        JOIN socioeconomic
            on LOWER(pop.county) = LOWER(socioeconomic.county)

        
    """
    ).fetchdf()

In [161]:
df.head(2)

,year,month,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,population,over21,propOver21,median_age,pct_male,store_count,county_1,fips,pct_less_than_9th_grade,less_than_9th_grade,pct_unemployed,unemployed,pct_below_poverty,below_poverty,pct_below_150pct_poverty,below_150pct_poverty,pct_families_below_poverty,families_below_poverty,pct_white,white,pct_hispanic,hispanic,pct_foreign_born,foreign_born,pct_black,black,pct_api,api,pct_aian,aian,pct_language_isolation,language_isolation,pct_less_than_high_school,less_than_high_school,pct_at_bachelor_degree,pct_high_school_lower
0,2023,2,FRANKLIN,0.16,0.52,0.11,0.08,0.13,42850.42,9895,7468,0.754725,42.6,0.513492,13,Franklin,19069.0,4.2,289.0,3.8,181.0,15.1,1481.0,24.5,2406.0,10.5,274.0,87.0,8679.0,16.1,1609.0,7.0,702.0,0.6,59.0,0.2,19.0,0.2,24.0,2.0,82.0,8.4,570.0,18.2,12.6
1,2023,3,SAC,0.14,0.50,0.04,0.13,0.19,61463.64,9663,7486,0.774708,44.5,0.503570,11,Sac,19161.0,2.8,194.0,3.4,176.0,7.9,757.0,16.5,1583.0,4.1,107.0,95.0,9267.0,4.1,398.0,3.1,303.0,0.6,54.0,1.3,128.0,0.0,4.0,1.5,62.0,7.8,541.0,25.2,10.6


Now encode month to use as a categorical feature:

In [162]:
months = pd.get_dummies(df.month, prefix='month', drop_first=True)
df = pd.concat([df.drop(columns='month'), months], axis=1)
df.head()

,year,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,population,over21,propOver21,median_age,pct_male,store_count,county_1,fips,pct_less_than_9th_grade,less_than_9th_grade,pct_unemployed,unemployed,pct_below_poverty,below_poverty,pct_below_150pct_poverty,below_150pct_poverty,pct_families_below_poverty,families_below_poverty,pct_white,white,pct_hispanic,hispanic,pct_foreign_born,foreign_born,pct_black,black,pct_api,api,pct_aian,aian,pct_language_isolation,language_isolation,pct_less_than_high_school,less_than_high_school,pct_at_bachelor_degree,pct_high_school_lower,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2023,FRANKLIN,0.16,0.52,0.11,0.08,0.13,42850.42,9895,7468,0.754725,42.6,0.513492,13,Franklin,19069.0,4.2,289.0,3.8,181.0,15.1,1481.0,24.5,2406.0,10.5,274.0,87.0,8679.0,16.1,1609.0,7.0,702.0,0.6,59.0,0.2,19.0,0.2,24.0,2.0,82.0,8.4,570.0,18.2,12.6,True,False,False,False,False,False,False,False,False,False,False
1,2023,SAC,0.14,0.50,0.04,0.13,0.19,61463.64,9663,7486,0.774708,44.5,0.503570,11,Sac,19161.0,2.8,194.0,3.4,176.0,7.9,757.0,16.5,1583.0,4.1,107.0,95.0,9267.0,4.1,398.0,3.1,303.0,0.6,54.0,1.3,128.0,0.0,4.0,1.5,62.0,7.8,541.0,25.2,10.6,False,True,False,False,False,False,False,False,False,False,False
2,2023,EMMET,0.24,0.39,0.05,0.14,0.18,77347.27,9277,7492,0.807589,43.8,0.514714,7,Emmet,19063.0,3.5,228.0,2.0,99.0,10.4,904.0,19.1,1663.0,6.9,153.0,88.6,8252.0,9.9,918.0,4.2,392.0,2.2,208.0,0.2,21.0,0.2,16.0,2.9,115.0,12.3,813.0,15.7,15.8,False,False,False,False,True,False,False,False,False,False,False
3,2023,SAC,0.20,0.45,0.05,0.11,0.18,80846.53,9663,7486,0.774708,44.5,0.503570,11,Sac,19161.0,2.8,194.0,3.4,176.0,7.9,757.0,16.5,1583.0,4.1,107.0,95.0,9267.0,4.1,398.0,3.1,303.0,0.6,54.0,1.3,128.0,0.0,4.0,1.5,62.0,7.8,541.0,25.2,10.6,False,False,False,False,True,False,False,False,False,False,False
4,2023,FLOYD,0.21,0.39,0.07,0.08,0.25,119442.68,15266,11772,0.771125,43.1,0.499607,11,Floyd,19067.0,2.6,282.0,3.5,273.0,11.4,1742.0,19.4,2973.0,6.7,263.0,90.4,14009.0,3.7,579.0,3.1,486.0,1.2,185.0,1.7,257.0,0.0,6.0,0.8,51.0,5.3,576.0,18.5,7.9,False,False,False,False,True,False,False,False,False,False,False


In [163]:
df = df.copy()
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df = df[df['revenue'] > 0]   
df.to_csv('../data/dead_model_data.csv', index=False)

## Linear Regression (from scratch)

First, the function to fit the model. It will take observed X and Y matrices and return the predictions and estimators.

In [23]:
def fit_lr(X, Y):
    
    X = np.column_stack([np.ones((X.shape[0], 1)), X.astype(float)]) # add column of ones
    B = np.linalg.pinv(X.T @ X) @ X.T @ Y

    return B

In [24]:
def predict_lr(X, B):
    
    X = np.column_stack([np.ones((X.shape[0], 1)), X.astype(float)]) # add column of ones
    Y_hat = X @ B

    return Y_hat

A cross validation function:

In [85]:
def lr_cross_val(data, features, response, k=5):

    # make k folds
    n = data.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    fold_size = n // k
    folds = [indices[i*fold_size:(i+1)*fold_size] for i in range(k-1)]
    folds.append(indices[(k-1)*fold_size:])  # last fold gets the remainder

    # loop over folds and store their predictions
    predictions = np.zeros(n)
    for fold_idx in folds:
        # training indices = everything not in this fold
        train_idx = np.setdiff1d(indices, fold_idx)
        test_idx = fold_idx

        train = data.iloc[train_idx]
        test = data.iloc[test_idx]

        # fit model
        B = fit_lr(train[features].values, train[response].values)

        # predict on test set
        Y_hat = predict_lr(test[features].values, B)

        predictions[test_idx] = Y_hat
    return predictions



Fit the model:

- *NOTE:* I first fit with all possible features, then after assessing model fit, feature importance, and multicollinearity (all below), I came back up and changed the features. So this is just the final model here, not all iterations.

In [164]:
df["store_ratio"] = df["store_count"] / df["over21"]
df['log_revenue'] = np.log(df['revenue'] / df['population'])

In [165]:
df.head()

,year,county,vodka_ptc,whiskey_ptc,tequila_ptc,rum_ptc,other_ptc,revenue,population,over21,propOver21,median_age,pct_male,store_count,county_1,fips,pct_less_than_9th_grade,less_than_9th_grade,pct_unemployed,unemployed,pct_below_poverty,below_poverty,pct_below_150pct_poverty,below_150pct_poverty,pct_families_below_poverty,families_below_poverty,pct_white,white,pct_hispanic,hispanic,pct_foreign_born,foreign_born,pct_black,black,pct_api,api,pct_aian,aian,pct_language_isolation,language_isolation,pct_less_than_high_school,less_than_high_school,pct_at_bachelor_degree,pct_high_school_lower,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,store_ratio,log_revenue
0,2023,FRANKLIN,0.16,0.52,0.11,0.08,0.13,42850.42,9895,7468,0.754725,42.6,0.513492,13,Franklin,19069.0,4.2,289.0,3.8,181.0,15.1,1481.0,24.5,2406.0,10.5,274.0,87.0,8679.0,16.1,1609.0,7.0,702.0,0.6,59.0,0.2,19.0,0.2,24.0,2.0,82.0,8.4,570.0,18.2,12.6,True,False,False,False,False,False,False,False,False,False,False,0.001741,1.465686
1,2023,SAC,0.14,0.50,0.04,0.13,0.19,61463.64,9663,7486,0.774708,44.5,0.503570,11,Sac,19161.0,2.8,194.0,3.4,176.0,7.9,757.0,16.5,1583.0,4.1,107.0,95.0,9267.0,4.1,398.0,3.1,303.0,0.6,54.0,1.3,128.0,0.0,4.0,1.5,62.0,7.8,541.0,25.2,10.6,False,True,False,False,False,False,False,False,False,False,False,0.001469,1.850142
2,2023,EMMET,0.24,0.39,0.05,0.14,0.18,77347.27,9277,7492,0.807589,43.8,0.514714,7,Emmet,19063.0,3.5,228.0,2.0,99.0,10.4,904.0,19.1,1663.0,6.9,153.0,88.6,8252.0,9.9,918.0,4.2,392.0,2.2,208.0,0.2,21.0,0.2,16.0,2.9,115.0,12.3,813.0,15.7,15.8,False,False,False,False,True,False,False,False,False,False,False,0.000934,2.120767
3,2023,SAC,0.20,0.45,0.05,0.11,0.18,80846.53,9663,7486,0.774708,44.5,0.503570,11,Sac,19161.0,2.8,194.0,3.4,176.0,7.9,757.0,16.5,1583.0,4.1,107.0,95.0,9267.0,4.1,398.0,3.1,303.0,0.6,54.0,1.3,128.0,0.0,4.0,1.5,62.0,7.8,541.0,25.2,10.6,False,False,False,False,True,False,False,False,False,False,False,0.001469,2.124249
4,2023,FLOYD,0.21,0.39,0.07,0.08,0.25,119442.68,15266,11772,0.771125,43.1,0.499607,11,Floyd,19067.0,2.6,282.0,3.5,273.0,11.4,1742.0,19.4,2973.0,6.7,263.0,90.4,14009.0,3.7,579.0,3.1,486.0,1.2,185.0,1.7,257.0,0.0,6.0,0.8,51.0,5.3,576.0,18.5,7.9,False,False,False,False,True,False,False,False,False,False,False,0.000934,2.057208


Fit the model:

- *NOTE:* I first fit with all possible features, then after assessing model fit, feature importance, and multicollinearity (all below), I came back up and changed the features. So this is just the final model here, not all iterations.

In [173]:
# 'pct_less_than_9th_grade', 'pct_less_than_high_school',
features = ['vodka_ptc', 'whiskey_ptc', 'tequila_ptc', 'rum_ptc',
       'other_ptc', 'propOver21',
       'median_age', 'pct_male', 'pct_at_bachelor_degree',
        'pct_unemployed',
       'pct_below_poverty', 'pct_high_school_lower',
       'pct_families_below_poverty','pct_white',
       'pct_hispanic', 'pct_foreign_born',
       'pct_black', 'pct_api', 'pct_aian',
       'pct_language_isolation',
       'month_5', 'month_6',
'month_11']
response = 'log_revenue'

# fit
Yhat = lr_cross_val(df, features, response, k=5)

 Assess model fit:

In [174]:
Y = df[response].values
X = df[features].values
RSS = np.sum((Y - Yhat)**2)
TSS = np.sum((Y - np.mean(Y))**2)
R2 = 1 - RSS/TSS
n, p = X.shape  # after adding intercept
adj_R2 = 1 - (1 - R2) * (n - 1) / (n - p)
MAE = np.mean(np.abs(Y - Yhat))
RMSE = np.sqrt(RSS / n)

print(f"R2 = {R2:.4f}")
print(f"Adjusted R2 = {adj_R2:.4f}")
print(f"MAE = {MAE:.2f}")
print(f"RMSE = {RMSE:.2f}")

R2 = 0.4780
Adjusted R2 = 0.4728
MAE = 0.29
RMSE = 0.37


Fit a model on all the data and look at the estimators:

In [175]:
# fit a final model on all the data
X = df[features]
Y = df[response]
B_final = fit_lr(X, Y)

In [176]:
coef_names = ['intercept'] + list(X.columns)
nonstd_df = pd.DataFrame({
    'feature': coef_names,
    'beta': B_final
})
nonstd_df = nonstd_df[nonstd_df['feature'] != 'intercept']
nonstd_df = nonstd_df.reindex(
    nonstd_df['beta'].abs().sort_values(ascending=False).index
)
print(nonstd_df)

                       feature      beta
8                     pct_male -9.048813
3                  tequila_ptc  5.999014
4                      rum_ptc  2.979270
5                    other_ptc  2.947500
6                   propOver21 -0.539675
2                  whiskey_ptc  0.507166
1                    vodka_ptc -0.172528
21                     month_5  0.133458
18                     pct_api  0.085897
22                     month_6  0.072987
16            pct_foreign_born -0.061606
17                   pct_black  0.051935
12       pct_high_school_lower -0.043332
15                pct_hispanic  0.038166
19                    pct_aian -0.035129
20      pct_language_isolation  0.028408
11           pct_below_poverty  0.024720
10              pct_unemployed  0.021150
7                   median_age  0.017794
23                    month_11  0.015557
13  pct_families_below_poverty -0.009397
9       pct_at_bachelor_degree -0.002153
14                   pct_white  0.001450


Check standardized coefficients because their magnitudes actually tell us importance:

In [177]:

X_std = X.apply(pd.to_numeric, errors='coerce').astype(float).std(axis=0, ddof=0).to_numpy()
Y_std = pd.to_numeric(Y, errors='coerce').astype(float).std(ddof=0)
beta_no_intercept = np.ravel(B_final[1:])  # shape (p,)
std_beta = beta_no_intercept * (X_std / Y_std)
importance = (pd.DataFrame({"feature": X.columns, "std_coef": std_beta})
              .sort_values("std_coef", key=np.abs, ascending=False)
              .reset_index(drop=True))

print(importance)


                       feature  std_coef
0        pct_high_school_lower -0.398000
1                 pct_hispanic  0.387989
2             pct_foreign_born -0.377454
3                  tequila_ptc  0.340758
4                    other_ptc  0.271895
5                      rum_ptc  0.232399
6                      pct_api  0.219961
7                    pct_black  0.158557
8            pct_below_poverty  0.157123
9                     pct_male -0.149998
10                  median_age  0.111405
11      pct_language_isolation  0.081662
12                     month_5  0.071419
13                 whiskey_ptc  0.062452
14                    pct_aian -0.049159
15  pct_families_below_poverty -0.048883
16              pct_unemployed  0.045669
17                     month_6  0.039058
18      pct_at_bachelor_degree -0.032172
19                  propOver21 -0.022734
20                   pct_white  0.015613
21                   vodka_ptc -0.014252
22                    month_11  0.008325


Check for collinearity between features:

In [178]:
# compute correlation matrix for numeric features
corr = X.corr().abs()

# show only the top correlated pairs
high_corr = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)
print("Most correlated feature pairs:\n")
print(high_corr.head(10))


Most correlated feature pairs:

pct_white          pct_foreign_born              0.908788
                   pct_hispanic                  0.831578
pct_foreign_born   pct_language_isolation        0.823051
pct_hispanic       pct_foreign_born              0.821608
pct_below_poverty  pct_families_below_poverty    0.817360
pct_hispanic       pct_language_isolation        0.792529
pct_white          pct_language_isolation        0.776324
pct_foreign_born   pct_api                       0.723608
pct_black          pct_api                       0.693088
pct_white          pct_api                       0.691832
dtype: float64
